In [ ]:
import pandas as pd
import plotly as plt
import plotly.express as px
import seaborn as sns
import numpy as np

In [ ]:
data = pd.read_csv('https://davmedia.test.tech-mail.ru/editor_uploads/2025/05/16/video_sessions_transformed.csv')

In [ ]:
data.head()

,user_id,video_owner_id,total_view_time,vk_platform,nav_screen,video_id
0,5137060586,197846393764,0,vk_video_ios,video_for_you,1524753638856
1,6791352918,255312742952,0,vk_video_android,video_search,1524750811524
2,8499858010,290290140528,2,vk_video_android,video_for_you,1524751697154
3,13064371614,296968444192,0,vk_video_android,video_search,1524751112304
4,15865765790,283074246524,0,vk_video_android,video_for_you,1524752278662


In [ ]:
data.shape

(4503, 6)

In [ ]:
data.describe()

,user_id,video_owner_id,total_view_time,video_id
count,4.503000e+03,4.503000e+03,4.503000e+03,4.503000e+03
mean,1.233544e+13,1.811593e+11,1.046599e+05,1.508005e+12
std,1.076879e+13,2.015803e+11,4.959592e+06,1.289130e+11
min,2.085183e+08,-1.122277e+12,-3.000000e+02,2.344280e+06
25%,2.248819e+12,1.377161e+11,0.000000e+00,1.524752e+12
50%,9.657196e+12,2.542896e+11,0.000000e+00,1.524754e+12
75%,2.048263e+13,2.839772e+11,2.000000e+00,1.524762e+12
max,3.729999e+13,2.649073e+12,2.405022e+08,1.525612e+12


In [ ]:
data.dtypes

,0
user_id,int64
video_owner_id,int64
total_view_time,int64
vk_platform,object
nav_screen,object
video_id,int64


## 1. Определение выбросов
### находим записи где, `total_view_time` меньше нуля

In [ ]:
fig = px.box(data, y='total_view_time')
fig.show()

In [ ]:
data_clean = data[data['total_view_time']>=0].copy() #убираем отрицательное время просмотра

### найдя выбросы на графике, убираем  их из dataframe на базе перцентиля

In [ ]:
upper_border = data['total_view_time'].quantile(0.9997)
data_clean = data_clean[(data_clean['total_view_time'] <= upper_border)].copy()

### находим также записи, где `video_owner_id` меньше нуля

In [ ]:
data_clean = data_clean[data_clean['video_owner_id']>=0].copy()

In [ ]:
### проверим платформы
data['vk_platform'].unique()

array(['vk_video_ios', 'vk_video_android', 'other'], dtype=object)

In [ ]:
data[data['vk_platform'] =='other']

,user_id,video_owner_id,total_view_time,vk_platform,nav_screen,video_id
4502,323234425,2394994,240502233,other,video_group_all,232412412


In [ ]:
data_clean = data_clean[data_clean['vk_platform'] !='other'].copy()

In [ ]:
### проверка валидности экранов смотрения
data['nav_screen'].unique()

array(['video_for_you', 'video_search', 'video_my_added',
       'video_my_history', 'video_group_all', 'video_group_main',
       'video_playlist', 'catalog_other', 'video_my_bookmarks', 'feed'],
      dtype=object)

In [ ]:
data_clean[data_clean['nav_screen'] == 'feed']

,user_id,video_owner_id,total_view_time,vk_platform,nav_screen,video_id


### после очистки выбросов по столбцам `total_view_time` и `vk_platform` экран смотрения  `feed` был удален

In [ ]:
data_clean.duplicated().unique() # дубликатов в датафрейме не обнаружено

array([False])

### 2. Рейтинг платформ с осознанными просмотрами

In [ ]:
platform_rate = data_clean.query('total_view_time >= 30').groupby('vk_platform').agg({'user_id' : 'count'})
platform_rate

,user_id
vk_platform,
vk_video_android,256
vk_video_ios,116


### наибольшее количество осознанных просмотров происходит на платформе vk_video_android

### 3. Определение топ10 сообществ по `total_view_time`

In [ ]:
video_owner_rate = data_clean.groupby('video_owner_id').agg({'total_view_time' : 'sum'}).sort_values('total_view_time', ascending=False).head(10)
video_owner_rate = video_owner_rate.reset_index()

### 4. Интересные выводы
## 1) Определение выбросов по полю `total_view_time` с помощью метода интерквартильного размаха даёт ложноположительные результаты по нахождению выбросов, так как среднее время просмотра равно 0 секунд. BoxPlot помог определить реальные выбросы в данных.
## 2) Около половины данных о просмотре видео имели значение `total_view_time` равное 0.
## 3) Выброс с самым большим количеством времени просмотра видео (`total_view_time`) имел значение параметра `экран смотрения` = `feed`. Возможно, это не выброс, а ошибка логирования и просмотры с ленты новостей просуммировались и попали в датасет.

In [ ]:
nav_screen_count = data_clean['nav_screen'].value_counts()
nav_screen_pct = (nav_screen_count / len(data_clean) * 100).round(2)
nav_screen_distribution = (pd.DataFrame({'count': nav_screen_count,'percentage': nav_screen_pct}).reset_index().rename(columns={'index': 'nav_screen'}))
nav_screen_distribution

,nav_screen,count,percentage
0,video_for_you,2192,51.56
1,video_search,1851,43.54
2,video_group_all,92,2.16
3,video_my_history,90,2.12
4,video_my_added,7,0.16
5,catalog_other,7,0.16
6,video_my_bookmarks,6,0.14
7,video_playlist,4,0.09
8,video_group_main,2,0.05


In [ ]:
fig = px.pie(
    nav_screen_distribution,
    names='nav_screen',
    values='count',
    title='Распределение сессий по экранам смотрения (nav_screen)',
    hole=0,
)
fig.show()